In [1]:
import os
import time
import psutil
import GPUtil
from functools import wraps

In [72]:
# !pip install nvidia-ml-py
from pynvml import *

In [73]:
def nvidia_info():
    # pip install nvidia-ml-py
    nvidia_dict = {
        "state": True,
        "nvidia_version": "",
        "nvidia_count": 0,
        "gpus": []
    }
    try:
        nvmlInit()
        nvidia_dict["nvidia_version"] = nvmlSystemGetDriverVersion()
        nvidia_dict["nvidia_count"] = nvmlDeviceGetCount()
        for i in range(nvidia_dict["nvidia_count"]):
            handle = nvmlDeviceGetHandleByIndex(i)
            memory_info = nvmlDeviceGetMemoryInfo(handle)
            gpu = {
                "gpu_name": nvmlDeviceGetName(handle),
                "total": memory_info.total,
                "free": memory_info.free,
                "used": memory_info.used,
                "temperature": f"{nvmlDeviceGetTemperature(handle, 0)}℃",
                "powerStatus": nvmlDeviceGetPowerState(handle)
            }
            nvidia_dict['gpus'].append(gpu)
    except NVMLError as _:
        nvidia_dict["state"] = False
    except Exception as _:
        nvidia_dict["state"] = False
    finally:
        try:
            nvmlShutdown()
        except:
            pass
    return nvidia_dict


def check_gpu_mem_usedRate():
    max_rate = 0.0
    while True:
        info = nvidia_info()
        # print(info)
        used = info['gpus'][0]['used']
        tot = info['gpus'][0]['total']
        print(f"GPU0 used: {used}, tot: {tot}, 使用率：{used/tot}")
        if used/tot > max_rate:
            max_rate = used/tot
        print("GPU0 最大使用率：", max_rate)


In [76]:
info = nvidia_info()
len(info['gpus'])

0

In [64]:
import time
import psutil
import GPUtil
from threading import Thread



class Monitor(Thread):
    def __init__(self, delay=1):
        super(Monitor, self).__init__()
        self.stopped = False
        self.delay = delay # Time between calls to GPUtil
        # self.start_time 
        # self.end_time
        self.time_points = []
        self.cpu_usage = []
        self.mem_usage = []
        self.start()

    def run(self):
        while not self.stopped:
            GPUtil.showUtilization()
            # Obtaining all the essential details
            self.time_points.append(time.time())
            self.cpu_usage.append(psutil.cpu_percent())
            self.mem_usage.append(psutil.virtual_memory().percent)
            time.sleep(self.delay)
        

    def stop(self):
        self.stopped = True
        return self.time_points, self.cpu_usage, self.mem_usage

In [37]:
def res_util(function):
    @wraps(function)
    def function_util(*args, **kwargs):
        start_time = time.time()
        cpu_usage = []
        mem_usage = []
        # cpu_usage.append(psutil.cpu_percent())
        # mem_usage.append(psutil.virtual_memory().percent)
        while not function.stopped:
            # Obtaining all the essential details
            cpu_usage.append(psutil.cpu_percent())
            mem_usage.append(psutil.virtual_memory().percent)
            print(cpu_usage)
            print(mem_usage)
    
            # Obtaining the GPU details
            GPUtil.showUtilization()
            time.sleep(1)
            
        end_time = time.time()
        total_time = end_time-start_time
        
        return function(*args, **kwargs), total_time, cpu_usage, mem_usage
    return function_util

In [29]:
import random

# @res_util
def random_sort(n):
    results = sorted([random.random() for i in range(n)])
    # random_sort.stopped = True
    return results

In [61]:
results, total_time, cpu_usage, mem_usage = random_sort(20000000)

KeyboardInterrupt: 

In [65]:
monitor = Monitor(1)
random_sort(20000000)
time_points, cpu_usage, mem_usage = monitor.stop()

| ID | GPU | MEM |
------------------
| ID | GPU | MEM |
------------------
| ID | GPU | MEM |
------------------
| ID | GPU | MEM |
------------------


In [66]:
time_points

[1700457428.6743805, 1700457429.804826, 1700457431.0107586, 1700457440.7123737]

In [67]:
cpu_usage

[0.1, 1.9, 1.9, 1.8]

In [68]:
mem_usage

[3.0, 3.1, 3.3, 3.0]

In [22]:
def funX(x):
    m_x = x
    funX.m_x = m_x

In [26]:
print(funX)

<function funX at 0x7f868c2e0430>


In [24]:
funX(3)

In [25]:
print("funX.m_x = ", funX.m_x) 

funX.m_x =  3
